In [4]:
# import all the libraries necessary
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# read the input file
loan_df = pd.read_csv('loan.csv')
# observe the warning, it signifies that the file is huge and there are non uniform data type in one column.
# we can ignore the warning
# I choose to fix it
# read the file again with dtype as object, there will be no warnings.
loan_df = pd.read_csv('loan.csv',dtype=object)


In [9]:
# let's look at the first 5 rows
loan_df.head(5)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN


In [12]:
# check the dimensions
print('Rows and Columns', loan_df.shape)
print(loan_df.info())
print(loan_df.columns)

Rows and Columns (39717, 111)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: object(111)
memory usage: 33.6+ MB
None
Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=111)


In [14]:

# we have 111 rows and 39717 columns.
# few rows have null values.
# we need to first see how many columns have only null data. They don't server any value to let's remove it
loan_df.isnull().sum()


id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
                              ...  
tax_liens                        39
tot_hi_cred_lim               39717
total_bal_ex_mort             39717
total_bc_limit                39717
total_il_high_credit_limit    39717
Length: 111, dtype: int64

In [15]:
# let's remove the columns that have all the null values
loan_df = loan_df.dropna(how='all', axis=1)
loan_df.shape

(39717, 57)

In [18]:
# let's remove the rows that have all the null values
print(loan_df.isnull().all(axis=1).sum())

0


In [28]:
# again check the data

# can we drop columns that have almost all values as null?
# we can do that based on the threshold

df1 = loan_df.dropna(axis=1, thresh=39700)
# df1= loan_df
df1.shape
# now see the columns


# all rows now have meaningful data

# we can check for duplicate entries as well. Any columns that seem duplicate or row duplicate?

(39717, 45)

In [35]:
l1 = set(df1.columns)

In [36]:
l2 = set(loan_df.columns)

In [37]:
l3 = list(l2 - l1)

In [38]:
print(l3)

{'tax_liens', 'desc', 'emp_length', 'collections_12_mths_ex_med', 'emp_title', 'mths_since_last_delinq', 'next_pymnt_d', 'chargeoff_within_12_mths', 'last_pymnt_d', 'revol_util', 'mths_since_last_record', 'pub_rec_bankruptcies'}
